## Connect to the MySQL database.

In [ ]:
import mysql.connector
from mysql.connector import Error

config = {
    'host': 'localhost',
    'database': 'database_name',
    'user': 'root',
    'password': 'pwd',
}

# 建立資料庫連線
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

# 執行 SELECT 指令選取表格的 data 欄位資料
query = ("SELECT data from garmin_push_log;")

# 執行 SELECT 指令
cursor.execute(query)

# 將資料匯出成 txt 檔案
with open('./output.txt', 'w') as f:
    for (data,) in cursor:
        f.write(data + '\n')

# 關閉資料庫連線
cursor.close()
cnx.close()

### read the data we export to the output.txt

In [ ]:
import json
import matplotlib.pyplot as plt

# read the data we export to the output.txt
data_path = './output.txt'
with open(data_path, "r") as input_file:
    content = input_file.read().splitlines()

In [ ]:
# capture the data concerining sleep.
result = []

for i in range(0, len(content)):

    try:
        data = json.loads(content[i])
        sleep_map = data.get('sleeps')[0].get('sleepLevelsMap')
        result.append(sleep_map)
    except:
        continue

# print(result)
print("Total map we have gotten:", str(len(result)))

"""
"""

# first, we test the specific index 
# we want to visualize.
number = 0
# print(result[number])

"""_comment_
@1chooo
I would recommend reworking the following 
code as it contains three nested for-loops 
that could potentially impact its efficiency. 
Additionally, it appears that we have only 
printed the first data figure obtained.
"""

# find the minimum of the TimeInSeconds
min_time = float('inf')
for phase in result[number].values():
    for period in phase:
        for key, value in period.items():
            if key.endswith('TimeInSeconds'):
                min_time = min(min_time, value)

# find the maximum of the TimeInSeconds
max_time = float('-inf')
for phase in result[number].values():
    for period in phase:
        for key, value in period.items():
            if key.endswith('TimeInSeconds'):
                max_time = max(max_time, value)

""" Show the result of sleep time. """
sleep_runtime = max_time - min_time
sleep_runtime_in_hr = sleep_runtime / 3_600

print("The minimum of the time:", str(min_time))
print("The minimum of the time:", str(max_time))
print("The runtime of the sleep time:", sleep_runtime, "(s) =", sleep_runtime_in_hr, "(hr)")


# let every TimeInSeconds - min_time
# to get the time era.
for phase in result[number].values():
    for period in phase:
        for key, value in period.items():
            if key.endswith('TimeInSeconds'):
                period[key] = value - min_time

# print(result[number])

#--------------------------------------------------------#

# get the interval of the time.
times = []

for phase in result[number].values():
    for period in phase:
        times.append(period['startTimeInSeconds'])
        times.append(period['endTimeInSeconds'])
times.sort()

start_time = times[0]
end_time = times[-1]

In [ ]:
"""_comment_
@1chooo
there is no present of the REM.
"""
# plot the figure.
fig, ax = plt.subplots()
phases = ['Deep', 'Light', 'Awake', 'REM']

for i, phase in enumerate(phases):

    y = i + 1

    for period in result[number].get(phase.lower(), []):
        start = period['startTimeInSeconds'] - start_time
        end = period['endTimeInSeconds'] - start_time
        duration = end - start
        ax.barh(y, duration, left=start, height=0.5, align='center', alpha=0.7, label=phase)

ax.set_yticks([1, 2, 3, 4])
ax.set_yticklabels(phases)
ax.set_xlabel('Time (s)')
ax.set_xlim(0, end_time - start_time)
ax.legend(loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.2))
plt.legend().set_visible(False)
plt.title('The cycle in one sleep.')

plt.show()

### Reference: 
[MySQL資料庫的安裝與基本操作](https://medium.com/web-design-zone/mysql%E8%B3%87%E6%96%99%E5%BA%AB%E7%9A%84%E5%AE%89%E8%A3%9D%E8%88%87%E5%9F%BA%E6%9C%AC%E6%93%8D%E4%BD%9C-f36a079afd85)

[SQL table資料表的基本操作](https://medium.com/web-design-zone/sql-table%E8%B3%87%E6%96%99%E8%A1%A8%E7%9A%84%E5%9F%BA%E6%9C%AC%E6%93%8D%E4%BD%9C-b7c0f830c60f)